In [1]:
import polars as pl
import numpy as np

from pickle import dump
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from scipy.spatial.transform import Rotation as R

import random
import time
import os

In [2]:
INPUT_DIR = "../../input/cmi-detect-behavior-with-sensor-data/"
OUTPUT_DIR = "../../models/debug/"

In [3]:
def handle_quaternion_missing_values(rot_data: np.ndarray) -> np.ndarray:
    """
    Handle missing values in quaternion data intelligently
    
    Key insight: Quaternions must have unit length |q| = 1
    If one component is missing, we can reconstruct it from the others
    """
    rot_cleaned = rot_data.copy()
    
    for i in range(len(rot_data)):
        row = rot_data[i]
        missing_count = np.isnan(row).sum()
        
        if missing_count == 0:
            # No missing values, normalize to unit quaternion
            norm = np.linalg.norm(row)
            if norm > 1e-8:
                rot_cleaned[i] = row / norm
            else:
                rot_cleaned[i] = [1.0, 0.0, 0.0, 0.0]  # Identity quaternion
                
        elif missing_count == 1:
            # One missing value, reconstruct using unit quaternion constraint
            # |w|² + |x|² + |y|² + |z|² = 1
            missing_idx = np.where(np.isnan(row))[0][0]
            valid_values = row[~np.isnan(row)]
            
            sum_squares = np.sum(valid_values**2)
            if sum_squares <= 1.0:
                missing_value = np.sqrt(max(0, 1.0 - sum_squares))
                # Choose sign for continuity with previous quaternion
                if i > 0 and not np.isnan(rot_cleaned[i-1, missing_idx]):
                    if rot_cleaned[i-1, missing_idx] < 0:
                        missing_value = -missing_value
                rot_cleaned[i, missing_idx] = missing_value
                rot_cleaned[i, ~np.isnan(row)] = valid_values
            else:
                rot_cleaned[i] = [1.0, 0.0, 0.0, 0.0]
        else:
            # More than one missing value, use identity quaternion
            rot_cleaned[i] = [1.0, 0.0, 0.0, 0.0]
    
    return rot_cleaned

# thanks: https://www.kaggle.com/code/nksusth/lb-0-78-quaternions-tf-bilstm-gru-attention
def calculate_angular_velocity_from_quat(rot_data, time_delta=1/10): # Assuming 10Hz sampling rate
    if isinstance(rot_data, pl.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].to_numpy()
    else:
        quat_values = rot_data

    num_samples = quat_values.shape[0]
    angular_vel = np.zeros((num_samples, 3))

    for i in range(num_samples - 1):
        q_t = quat_values[i]
        q_t_plus_dt = quat_values[i+1]

        if np.all(np.isnan(q_t)) or np.all(np.isclose(q_t, 0)) or \
           np.all(np.isnan(q_t_plus_dt)) or np.all(np.isclose(q_t_plus_dt, 0)):
            continue

        try:
            rot_t = R.from_quat(q_t)
            rot_t_plus_dt = R.from_quat(q_t_plus_dt)

            # Calculate the relative rotation
            delta_rot = rot_t.inv() * rot_t_plus_dt
            
            # Convert delta rotation to angular velocity vector
            # The rotation vector (Euler axis * angle) scaled by 1/dt
            # is a good approximation for small delta_rot
            angular_vel[i, :] = delta_rot.as_rotvec() / time_delta
        except ValueError:
            # If quaternion is invalid, angular velocity remains zero
            pass
            
    return angular_vel

# thanks: https://www.kaggle.com/code/nksusth/lb-0-78-quaternions-tf-bilstm-gru-attention
def calculate_angular_distance(rot_data):
    if isinstance(rot_data, pl.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].to_numpy()
    else:
        quat_values = rot_data

    num_samples = quat_values.shape[0]
    angular_dist = np.zeros(num_samples)

    for i in range(num_samples - 1):
        q1 = quat_values[i]
        q2 = quat_values[i+1]

        if np.all(np.isnan(q1)) or np.all(np.isclose(q1, 0)) or \
           np.all(np.isnan(q2)) or np.all(np.isclose(q2, 0)):
            angular_dist[i] = 0 # Или np.nan, в зависимости от желаемого поведения
            continue
        try:
            # Преобразование кватернионов в объекты Rotation
            r1 = R.from_quat(q1)
            r2 = R.from_quat(q2)

            # Вычисление углового расстояния: 2 * arccos(|real(p * q*)|)
            # где p* - сопряженный кватернион q
            # В scipy.spatial.transform.Rotation, r1.inv() * r2 дает относительное вращение.
            # Угол этого относительного вращения - это и есть угловое расстояние.
            relative_rotation = r1.inv() * r2
            
            # Угол rotation vector соответствует угловому расстоянию
            # Норма rotation vector - это угол в радианах
            angle = np.linalg.norm(relative_rotation.as_rotvec())
            angular_dist[i] = angle
        except ValueError:
            angular_dist[i] = 0 # В случае недействительных кватернионов
            pass
            
    return angular_dist

def preprocess_left_handed(l_tr):
    rot_cleaned = handle_quaternion_missing_values(l_tr[["rot_w","rot_x", "rot_y", "rot_z"]].to_numpy())
    rot_scipy = rot_cleaned[:, [1, 2, 3, 0]]
    
    norms = np.linalg.norm(rot_scipy, axis=1)
    if np.any(norms < 1e-8):
        # Replace problematic quaternions with identity
        mask = norms < 1e-8
        rot_scipy[mask] = [0.0, 0.0, 0.0, 1.0]  # Identity quaternion in scipy format
        print("yes")
    
    r = R.from_quat(rot_scipy)
    tmp = r.as_euler("xyz")
    tmp[:,1] = - tmp[:,1]
    tmp[:,2] = - tmp[:,2]
    r = R.from_euler("xyz", tmp)
    tmp = r.as_quat()
    l_tr = l_tr.with_columns(pl.DataFrame(tmp, schema=["rot_x", "rot_y", "rot_z", "rot_w"]))
    l_tr = l_tr.with_columns(-pl.col("acc_x"))
    
    tmp = l_tr[["thm_3", "thm_5"]]
    tmp.columns = ["thm_5", "thm_3"]
    l_tr = l_tr.with_columns(tmp)
    
    swap_1_2_4_base = [[0,7],[1,6],[2,5],[3,4], [4,3], [5,2],[6,1],[7,0]]
    swap_3_5_base = [[0,56],[8,48],[16,40], [24,32],[32,24], [40,16],[48,8], [56,0]]
    
    swap_1_2_4 = list()
    for i in range(0,64,8):
        ll = list()
        for (k,l) in swap_1_2_4_base:
            ll.append([k+i, l+i])
        swap_1_2_4 += ll
    
    swap_3_5 = list()
    for i in range(8):
        ll = list()
        for (k,l) in swap_3_5_base:
            ll.append([k+i, l+i])
        swap_3_5 += ll
    
    l_df = l_tr
    
    for (k,l) in zip(["tof_3_v" + str(x) for x in range(64)], ["tof_5_v" + str(x) for x in range(64)]):
        l_tr = l_tr.with_columns(l_df[k].alias(l))
    
    for (k,l) in zip(["tof_3_v" + str(x) for x in range(64)], ["tof_5_v" + str(x) for x in range(64)]):
        l_tr = l_tr.with_columns(l_df[l].alias(k))
    
    l_df = l_tr
    
    for i in [1,2,4]:
        for (k, l) in swap_1_2_4:
            l_tr = l_tr.with_columns(l_df["tof_" + str(i) + "_v"+str(k)].alias("tof_" + str(i) + "_v"+str(l)))
    
    for i in [3,5]:
        for (k, l) in swap_3_5:
            l_tr = l_tr.with_columns(l_df["tof_" + str(i) + "_v"+str(k)].alias("tof_" + str(i) + "_v"+str(l)))
    return l_tr

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
device

device(type='cuda', index=0)

In [6]:
tr = pl.read_csv(os.path.join(INPUT_DIR,"train.csv"))
no_gesture = 'SEQ_011975'
tr = tr.filter(pl.col("sequence_id") != no_gesture)

#null_seqs = pl.read_csv("../input/null_seqs.csv")
#tr = tr.filter(~pl.col("sequence_id").is_in(null_seqs[:,0].implode()))

demo = pl.read_csv(os.path.join(INPUT_DIR,"train_demographics.csv"))

In [7]:
r_subjs = demo.filter(pl.col("handedness") == 1)["subject"].to_list()
l_subjs = demo.filter(pl.col("handedness") == 0)["subject"].to_list()

r_subjs.remove("SUBJ_019262") # these subjects wore the device on the wrong side of the wrist and score worse then left handers.
r_subjs.remove("SUBJ_045235") # further, tof and thm are probably not salvagable, but maybe imu can be adjusted, could look into later

r_new = tr.filter(pl.col("subject").is_in(r_subjs))
l_new = tr.filter(pl.col("subject").is_in(l_subjs))

l_new = preprocess_left_handed(l_new)
    
tr = pl.concat([r_new, l_new])
tr = tr.sort(by="row_id")

In [8]:
labels = tr.filter(pl.col("sequence_counter") ==0)[:,["sequence_id", "subject","gesture"]]
labels = labels.sort(by="gesture")
label_encoder = LabelEncoder()
labels = labels.with_columns(pl.Series(label_encoder.fit_transform(labels["gesture"])).alias("gesture"))
oh_encoder = OneHotEncoder()
onehotted = oh_encoder.fit_transform(pl.DataFrame(labels["gesture"])).toarray()
onehotted = pl.DataFrame(onehotted, orient="row", schema=label_encoder.classes_.tolist())
labels = labels.hstack(onehotted)
labels_df = labels.sort(by="sequence_id")

with open(os.path.join(OUTPUT_DIR,"encoder.pkl"), "wb") as f:
    dump(label_encoder, f)

In [9]:
dfs = list()
for (idx, df) in tr.group_by("sequence_id", maintain_order = True):
    df = df.with_columns(pl.col("acc_x").diff().alias("diff_x"))
    df = df.with_columns(pl.col("acc_y").diff().alias("diff_y"))
    df = df.with_columns(pl.col("acc_z").diff().alias("diff_z"))
    df = df.with_columns((pl.col("acc_x").pow(2) + pl.col("acc_y").pow(2) + pl.col("acc_z").pow(2)).sqrt().alias("mag"))
    df = df.with_columns((2*pl.col("rot_w").arccos()).alias("angle"))
    
    ang_vel = calculate_angular_velocity_from_quat(df)
    ang_dist = calculate_angular_distance(df)
    df = df.with_columns(pl.DataFrame(ang_vel, schema=["ang_vel_x", "ang_vel_y", "ang_vel_z"]))
    df = df.with_columns(pl.Series(ang_dist).alias("ang_dist"))
    
    dfs.append(df)
tr = pl.concat(dfs)

In [10]:
scaler = RobustScaler()

In [11]:
imu_cols = ["acc_x","acc_y", "acc_z", "rot_w", "rot_x", "rot_y", "rot_z"]
diff_cols = ["diff_x", "diff_y", "diff_z"]
extra_imu = ["mag", "angle"]
extra_extra_imu = ["ang_vel_x", "ang_vel_y", "ang_vel_z", "ang_dist"]

In [12]:
target_cols = imu_cols+diff_cols + extra_imu + extra_extra_imu

In [13]:
len(target_cols)

16

In [14]:
def prep_data(tr_df, labels_df, target_cols, scaler):

    tr_df = tr_df.with_columns(pl.col(target_cols).replace([float("inf"), float("-inf"), float("nan")], None))
    scaled = pl.DataFrame(scaler.fit_transform(tr_df[target_cols]), schema=target_cols)
    tr_df = tr_df.with_columns(scaled)
    
    tr_df = tr_df.with_columns(pl.col(target_cols).replace([float("inf"), float("-inf"), float("nan")], None))
    tr_df =tr_df.fill_null(0)
    
    seq_length = 128
    x_tr = np.zeros((tr_df.filter(pl.col("sequence_counter")  == 0).shape[0], len(target_cols), seq_length))
    for e, (idx, df) in enumerate(tr_df.group_by("sequence_id", maintain_order=True)):
        x_tr[e, :,-df.shape[0]:] = df[target_cols].to_numpy()[-seq_length:,:].T
    
    y_tr = labels_df[:,3:].to_numpy()
    
    return x_tr, y_tr, scaler

In [15]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data, targets, noise_factor=120):
        self.data = data
        self.targets = torch.Tensor(targets)
        self.noise_factor = noise_factor
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        
        if self.noise_factor:
            r = np.random.rand(x.shape[0], x.shape[1])/self.noise_factor - 1/(2*self.noise_factor)
            x = x+r
            x = x.astype(np.float32)
        
        r = np.random.uniform(0.75,1.5)
        x[:3,:] = x[:3,:]*r
        x[7:,:] = x[7:,:]*r
        
        return x, y
    
    def __len__(self):
        return len(self.data)


class TestTimeDataset(torch.utils.data.Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = torch.Tensor(targets)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        
        return x, y
    
    def __len__(self):
        return len(self.data)


In [16]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding="same")
        self.bn1 = nn.BatchNorm1d(num_features=out_channels)
        self.conv2 = nn.Conv1d(in_channels=out_channels, out_channels=out_channels, kernel_size=kernel_size, padding="same")
        self.bn2 = nn.BatchNorm1d(num_features=out_channels)
        
        if in_channels != out_channels:
            self.res_layer = torch.nn.Conv1d(in_channels, out_channels, kernel_size=1, padding="same")
        else:
            self.res_layer = None
            
    def forward(self, x):
        res = x
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        if self.res_layer is None:
            x = torch.add(x, res)
        else: 
            x = torch.add(x, self.res_layer(res))
        return x 

class SEBlock(nn.Module):
    def __init__(self, channels, reduction=8):
        super().__init__()
        self.squeeze = nn.AdaptiveAvgPool1d(1)
        self.excitation = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        b, c, _ = x.size()
        y = self.squeeze(x).view(b, c)
        y = self.excitation(y).view(b, c, 1)
        return x * y.expand_as(x)

class ResidualBlockSE(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding="same")
        self.bn1 = nn.BatchNorm1d(num_features=out_channels)
        self.conv2 = nn.Conv1d(in_channels=out_channels, out_channels=out_channels, kernel_size=kernel_size, padding="same")
        self.bn2 = nn.BatchNorm1d(num_features=out_channels)
        self.se = SEBlock(out_channels)
        
        if in_channels != out_channels:
            self.res_layer = nn.Conv1d(in_channels, out_channels, kernel_size=1, padding="same")
            self.res_bn = nn.BatchNorm1d(out_channels)
        else:
            self.res_layer = None
            self.res_bn = None
            
    def forward(self, x):
        res = x
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x = self.se(x)
        if self.res_layer is None:
            x = torch.add(x, res)
        else: 
            x = torch.add(x, self.res_bn(self.res_layer(res)))
        x = F.relu(x)
        return x     

In [17]:
 class RNNet(nn.Module):
    def __init__(self, chan=64):
        super().__init__()

        self.block0_0 = ResidualBlock(4, chan,3)
        self.block0_1 = ResidualBlock(chan, chan,3)
        self.block0_2 = ResidualBlock(chan, chan, 3)

        self.block1_0 = ResidualBlock(5, chan,3)
        self.block1_1 = ResidualBlock(chan, chan,3)
        self.block1_2 = ResidualBlock(chan, chan, 3)

        self.block2_0 = ResidualBlock(3, chan,3)
        self.block2_1 = ResidualBlock(chan, chan,3)
        self.block2_2 = ResidualBlock(chan, chan, 3)

        self.block3_0 = ResidualBlock(4, chan,3)
        self.block3_1 = ResidualBlock(chan, chan,3)
        self.block3_2 = ResidualBlock(chan, chan, 3)
        
        self.rnn = nn.GRU(4*chan, 128,
            num_layers = 2, 
            batch_first =True,
            bidirectional=True)

        self.fc0 = nn.Linear(256, 256)
        self.drop = nn.Dropout(0.2)
        self.fc1 = nn.Linear(256, 18)
                    
    def forward(self, x):
        in1 = torch.cat([x[:,:3,:], x[:,10:11,:]], dim=1)
        in2 = torch.cat([x[:,3:7,:], x[:,11:12,:]], dim=1)
        in3 = x[:,7:10,:]
        in4 = x[:,12:,:]
        x0 = self.block0_2(self.block0_1(self.block0_0(in1)))
        x1 = self.block1_2(self.block1_1(self.block1_0(in2)))
        x2 = self.block2_2(self.block2_1(self.block2_0(in3)))
        x3 = self.block3_2(self.block3_1(self.block3_0(in4)))
        
        x = torch.cat([x0,x1,x2,x3], dim=1)
        
        x = x.permute(0, 2, 1)
        x,_ = self.rnn(x)
        x = self.drop(F.relu(self.fc0(x)))
        x = self.fc1(x)
        x = x[:,-30:,:].mean(axis=1)
        return x 

 class RNNetSE(nn.Module):
    def __init__(self, chan=64):
        super().__init__()

        self.block0_0 = ResidualBlockSE(4, chan,3)
        self.block0_1 = ResidualBlockSE(chan, chan,3)
        self.block0_2 = ResidualBlockSE(chan, chan, 3)

        self.block1_0 = ResidualBlockSE(5, chan,3)
        self.block1_1 = ResidualBlockSE(chan, chan,3)
        self.block1_2 = ResidualBlockSE(chan, chan, 3)

        self.block2_0 = ResidualBlockSE(3, chan,3)
        self.block2_1 = ResidualBlockSE(chan, chan,3)
        self.block2_2 = ResidualBlockSE(chan, chan, 3)

        self.block3_0 = ResidualBlockSE(4, chan,3)
        self.block3_1 = ResidualBlockSE(chan, chan,3)
        self.block3_2 = ResidualBlockSE(chan, chan, 3)
        
        self.rnn = nn.GRU(4*chan, 128,
            num_layers = 2, 
            batch_first =True,
            bidirectional=True)

        self.fc0 = nn.Linear(256, 256)
        self.drop = nn.Dropout(0.2)
        self.fc1 = nn.Linear(256, 18)
                    
    def forward(self, x):
        in1 = torch.cat([x[:,:3,:], x[:,10:11,:]], dim=1)
        in2 = torch.cat([x[:,3:7,:], x[:,11:12,:]], dim=1)
        in3 = x[:,7:10,:]
        in4 = x[:,12:,:]
        x0 = self.block0_2(self.block0_1(self.block0_0(in1)))
        x1 = self.block1_2(self.block1_1(self.block1_0(in2)))
        x2 = self.block2_2(self.block2_1(self.block2_0(in3)))
        x3 = self.block3_2(self.block3_1(self.block3_0(in4)))
        
        x = torch.cat([x0,x1,x2,x3], dim=1)
        
        x = x.permute(0, 2, 1)
        x,_ = self.rnn(x)
        x = self.drop(F.relu(self.fc0(x)))
        x = self.fc1(x)
        x = x[:,-30:,:].mean(axis=1)
        return x 



class Net(nn.Module):
    def __init__(self,n_blocks=2, chan=64, kern=5, kern_increase=4):
        super().__init__()
        
        self.block0_0 = ResidualBlock(4, chan,3)
        self.block0_1 = ResidualBlock(chan, chan,3)
        self.block0_2 = ResidualBlock(chan, chan, 3)

        self.block1_0 = ResidualBlock(5, chan,3)
        self.block1_1 = ResidualBlock(chan, chan,3)
        self.block1_2 = ResidualBlock(chan, chan, 3)

        self.block2_0 = ResidualBlock(3, chan,3)
        self.block2_1 = ResidualBlock(chan, chan,3)
        self.block2_2 = ResidualBlock(chan, chan, 3)

        self.block3_0 = ResidualBlock(4, chan,3)
        self.block3_1 = ResidualBlock(chan, chan,3)
        self.block3_2 = ResidualBlock(chan, chan, 3)
        
        self.layers = nn.ModuleList()
        for i in range(n_blocks+1):
            if i == 0:
                self.layers.append(ResidualBlock(4*chan, 4*chan,kern))
            else:
                self.layers.append(ResidualBlock((4*chan)*2**((i-1)//2), (4*chan)*2**(i//2),kern+kern_increase*i))
        self.pool = nn.MaxPool1d(2,2)  
        self.fc0 = nn.Linear((4*chan)*2**(i//2), 18)
        
    def forward(self, x):
        in1 = torch.cat([x[:,:3,:], x[:,10:11,:]], dim=1)
        in2 = torch.cat([x[:,3:7,:], x[:,11:12,:]], dim=1)
        in3 = x[:,7:10,:]
        in4 = x[:,12:,:]
        x0 = self.block0_2(self.block0_1(self.block0_0(in1)))
        x1 = self.block1_2(self.block1_1(self.block1_0(in2)))
        x2 = self.block2_2(self.block2_1(self.block2_0(in3)))
        x3 = self.block3_2(self.block3_1(self.block3_0(in4)))
        
        x = torch.cat([x0,x1,x2,x3], dim=1)
        
        for i in range(len(self.layers)-1):
            x = self.pool(self.layers[i](x))
        x = self.layers[-1](x)
        x = torch.mean(x, dim=-1)
        x = self.fc0(x)
        return x 

class NetSE(nn.Module):
    def __init__(self,n_blocks=2, chan=64, kern=5, kern_increase=4):
        super().__init__()
        
        self.block0_0 = ResidualBlockSE(4, chan,3)
        self.block0_1 = ResidualBlockSE(chan, chan,3)
        self.block0_2 = ResidualBlockSE(chan, chan, 3)

        self.block1_0 = ResidualBlockSE(5, chan,3)
        self.block1_1 = ResidualBlockSE(chan, chan,3)
        self.block1_2 = ResidualBlockSE(chan, chan, 3)

        self.block2_0 = ResidualBlockSE(3, chan,3)
        self.block2_1 = ResidualBlockSE(chan, chan,3)
        self.block2_2 = ResidualBlockSE(chan, chan, 3)

        self.block3_0 = ResidualBlockSE(4, chan,3)
        self.block3_1 = ResidualBlockSE(chan, chan,3)
        self.block3_2 = ResidualBlockSE(chan, chan, 3)
        
        self.layers = nn.ModuleList()
        for i in range(n_blocks+1):
            if i == 0:
                self.layers.append(ResidualBlockSE(4*chan, 4*chan,kern))
            else:
                self.layers.append(ResidualBlockSE((4*chan)*2**((i-1)//2), (4*chan)*2**(i//2),kern+kern_increase*i))
        self.pool = nn.MaxPool1d(2,2)  
        self.fc0 = nn.Linear((4*chan)*2**(i//2), 18)
        
    def forward(self, x):
        in1 = torch.cat([x[:,:3,:], x[:,10:11,:]], dim=1)
        in2 = torch.cat([x[:,3:7,:], x[:,11:12,:]], dim=1)
        in3 = x[:,7:10,:]
        in4 = x[:,12:,:]
        x0 = self.block0_2(self.block0_1(self.block0_0(in1)))
        x1 = self.block1_2(self.block1_1(self.block1_0(in2)))
        x2 = self.block2_2(self.block2_1(self.block2_0(in3)))
        x3 = self.block3_2(self.block3_1(self.block3_0(in4)))
        
        x = torch.cat([x0,x1,x2,x3], dim=1)
        
        for i in range(len(self.layers)-1):
            x = self.pool(self.layers[i](x))
        x = self.layers[-1](x)
        x = torch.mean(x, dim=-1)
        x = self.fc0(x)
        return x 

# https://www.kaggle.com/code/takoihiraokazu/cv-ensemble-sub-0607-1
class RnnModelImu(nn.Module):
    def __init__(
        self, dropout=0.2,
        input_numerical_size=16,
        numeraical_linear_size = 128,
        model_size = 128,
        linear_out = 128,
        out_size=18):
        super(RnnModelImu, self).__init__()
        self.numerical_linear  = nn.Sequential(
                nn.Linear(input_numerical_size, numeraical_linear_size),
                nn.LayerNorm(numeraical_linear_size)
            )
        
        self.rnn = nn.GRU(numeraical_linear_size, model_size,
                            num_layers = 2, 
                            bidirectional=True,
                            batch_first=True)
        self.linear_out  = nn.Sequential(
                nn.Linear(model_size*2, 
                          linear_out),
                nn.LayerNorm(linear_out),
                nn.Dropout(dropout),
                nn.Linear(linear_out, 
                          out_size))
        self._reinitialize()
        
    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        """
        for name, p in self.named_parameters():
            if 'rnn' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
    
    def forward(self, numerical_array):
        numerical_array = numerical_array.permute(0, 2, 1)
        numerical_embedding = self.numerical_linear(numerical_array)
        output,_ = self.rnn(numerical_embedding)
        output = self.linear_out(output)
        return output[:,-30:,:].mean(axis=1)


In [18]:
len(target_cols)

16

In [20]:
x_tr, y_tr, scaler = prep_data(tr, labels_df, target_cols, scaler)

with open(os.path.join(OUTPUT_DIR,"imuonly_scaler.pkl"), "wb") as f:
    dump(scaler, f)

In [21]:
start = time.time()

In [22]:
seed = 20
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed) 

In [23]:
for p in range(40):   
    net = RNNet().to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(net.parameters(), lr=0.0005)
    
    tr_dataset = MyDataset(x_tr.astype(np.float32), y_tr.astype(np.float32))
    trainloader = torch.utils.data.DataLoader(tr_dataset, batch_size=64,
                                              shuffle=True, num_workers=4)

    for epoch in range(60): 
        net.train()
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
    
            optimizer.zero_grad()
    
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
           

    model_state = net.state_dict()
    torch.save(model_state, os.path.join(OUTPUT_DIR,f"imuonly_cnngru128_itr{p}"))
    print(p)
    

0


In [24]:
for p in range(40):   
    net = Net().to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(net.parameters(), lr=0.0005)
    
    tr_dataset = MyDataset(x_tr.astype(np.float32), y_tr.astype(np.float32))   
    trainloader = torch.utils.data.DataLoader(tr_dataset, batch_size=64,
                                              shuffle=True, num_workers=4)

    itr = 0
    for epoch in range(80):
        net.train()
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
    
            optimizer.zero_grad()
    
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()


    model_state = net.state_dict()
    torch.save(model_state, os.path.join(OUTPUT_DIR,f"imuonly_cnn64chan_itr{p}"))
    print(p)
    

0


In [25]:
for p in range(40):   
    net = RnnModelImu().to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(net.parameters(), lr=0.0005)
    
    tr_dataset = MyDataset(x_tr.astype(np.float32), y_tr.astype(np.float32))
    trainloader = torch.utils.data.DataLoader(tr_dataset, batch_size=64,
                                              shuffle=True, num_workers=4)

    itr = 0
    for epoch in range(60): 
        net.train()
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    

    model_state = net.state_dict()
    torch.save(model_state, os.path.join(OUTPUT_DIR,f"imuonly_densegru_itr{p}"))
    print(p)

0


In [26]:
for p in range(40):   
    net = RNNetSE().to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(net.parameters(), lr=0.0005)
    
    tr_dataset = MyDataset(x_tr.astype(np.float32), y_tr.astype(np.float32))
    trainloader = torch.utils.data.DataLoader(tr_dataset, batch_size=64,
                                              shuffle=True, num_workers=4)

    for epoch in range(60): 
        net.train()
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
    
            optimizer.zero_grad()
    
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
           

    model_state = net.state_dict()
    torch.save(model_state, os.path.join(OUTPUT_DIR,f"imuonly_cnngru128SE_itr{p}"))
    print(p)

0


In [27]:
for p in range(40):   
    net = NetSE().to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(net.parameters(), lr=0.0005)
    
    tr_dataset = MyDataset(x_tr.astype(np.float32), y_tr.astype(np.float32))   
    trainloader = torch.utils.data.DataLoader(tr_dataset, batch_size=64,
                                              shuffle=True, num_workers=4)

    itr = 0
    for epoch in range(80):
        net.train()
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
    
            optimizer.zero_grad()
    
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()


    model_state = net.state_dict()
    torch.save(model_state, os.path.join(OUTPUT_DIR,f"imuonly_cnn64chanSE_itr{p}"))
    print(p)

0


In [28]:
print((time.time()-start)/60/60)

0.033852232893308
